In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Uploading over 8500 pictures

# Create a folder to extract images into
!mkdir -p /content/unzipped_images1/

# Unzip your file (using your exact path)
!unzip "/content/drive/MyDrive/new_images_folder.zip" -d /content/unzipped_images1/

In [ ]:
# Verify the files were extracted
import os
extracted_files = os.listdir("/content/unzipped_images1/new_images_folder")
print(f"Found {len(extracted_files)} files/folders in the unzipped directory")
print("First 5 items:", extracted_files[:5])

In [ ]:
# Uploading libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid, softmax
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
from tensorflow.keras import Input
from tensorflow.keras.regularizers import l2

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [ ]:
#Flattening the pictures and creating labels
#Will be used for loading date

import os
import numpy as np
from PIL import Image

image_folder = '/content/unzipped_images1/new_images_folder'
output_data_file = 'new_deck_flatten1.npy'
output_labels_file = 'new_card_labels1.npy'
image_size = (128, 128)
cards = ['ace_spades', '2_spades', '3_spades', '4_spades', '5_spades',
         '6_spades', '7_spades', '8_spades', '9_spades', '10_spades',
         'jack_spades', 'queen_spades', 'king_spades',
         'ace_hearts', '2_hearts', '3_hearts', '4_hearts', '5_hearts',
         '6_hearts', '7_hearts', '8_hearts', '9_hearts', '10_hearts',
         'jack_hearts', 'queen_hearts', 'king_hearts',
         'ace_diamonds', '2_diamonds', '3_diamonds', '4_diamonds', '5_diamonds',
         '6_diamonds', '7_diamonds', '8_diamonds', '9_diamonds', '10_diamonds',
         'jack_diamonds', 'queen_diamonds', 'king_diamonds',
         'ace_clubs', '2_clubs', '3_clubs', '4_clubs', '5_clubs',
         '6_clubs', '7_clubs', '8_clubs', '9_clubs', '10_clubs',
         'jack_clubs', 'queen_clubs', 'king_clubs',]

all_rows = []
all_labels = []

for card in cards:
    image_files = sorted([
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.startswith(card)
    ])

    for file in image_files:
        img = Image.open(file).convert('RGB')
        img = img.resize(image_size)
        img_array = np.array(img)

        for k in [0, 1, 2, 3]:  # original + 3 rotations
            rotated = np.rot90(img_array, k=k)
            flat = rotated.flatten()
            all_rows.append(flat)
            all_labels.append(card)  # Add label for each rotation

# Convert to arrays
data_array = np.stack(all_rows)                       # Shape: (4160, H*W*3)
labels_array = np.array(all_labels).reshape(-1, 1)    # Shape: (4160, 1)

# Save both
np.save(output_data_file, data_array)
np.save(output_labels_file, labels_array)

print(f"Saved data to {output_data_file} with shape {data_array.shape}")
print(f"Saved labels to {output_labels_file} with shape {labels_array.shape}")


In [ ]:
# Loading the data
from public_tests import *

from autils import *
from lab_utils_softmax import plt_softmax
np.set_printoptions(precision=2)

X, y = load_data()

In [ ]:
# Reshaping
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

X_cnn = X.reshape(-1, 128, 128, 3)
X_cnn = X_cnn/255.0

In [ ]:
# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  # Cards shouldn't be flipped horizontally
    fill_mode='nearest',
    validation_split=0.1
)

In [ ]:
# Model definition
model = Sequential([
    Input(shape=(128,128,3)),
    Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(52, activation='softmax')
])

In [ ]:
# Loading y-labels and converting to integers
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load string labels
y_labels = np.load("new_card_labels1.npy")  # contains strings like "king_hearts", etc.
y_labels = y_labels.ravel()

# Convert to integer class IDs
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_labels)

In [ ]:
# Shuffling the cards/data and its labels together

from sklearn.utils import shuffle

X_cnn, y_encoded = shuffle(X_cnn, y_encoded, random_state=42)

In [ ]:
# Splitting the data

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_cnn, y_encoded,
    test_size=0.1,
    stratify=y_encoded,
    random_state=42
)

# Then create separate generators
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # No augmentation for validation

train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=32
)

val_generator = val_datagen.flow(
    X_val, y_val,
    batch_size=32
)

In [ ]:
# Training
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Lower learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

import pickle

def save_history(history, filename='training_history.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(history.history, f)

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    'new_best_model.h5',             # File to save
    monitor='val_accuracy',      # Metric to track
    save_best_only=True,         # Only save if val_accuracy improves
    mode='max',                  # Maximize val_accuracy
    verbose=1                    # Show messages when saving
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[checkpoint],      # Checkpoint here
    verbose=1
)